In [32]:
from randomuser import RandomUser
import pandas as pd
import requests
import random
from IPython.utils import io
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from pathlib import Path
from dotenv import load_dotenv
import os
from googleapiclient.http import MediaFileUpload



In [33]:
"""
Tarefa 1

Usar a API do Random User Generator para gerar uma lista de 1.000 usuários brasileiros
"""

users = RandomUser.generate_users(10, {"nat": "br"})

print(len(users))

10


In [34]:
"""
Tarefa 1.1

Faça uma tabela com todas as informações desses usuários
"""

users_info = []
for user in users:
    user_info = {
        "first_name": user.get_first_name(),
        "last_name": user.get_last_name(),
        "gender": user.get_gender(),
        "email": user.get_email(),
        "cell": user.get_cell(),
        "age": user.get_age(),
    }
    users_info.append(user_info)

users_df = pd.DataFrame(users_info)

print(users_df.head().to_markdown())


|    | first_name   | last_name   | gender   | email                           | cell           |   age |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|
|  0 | Jovina       | Melo        | female   | jovina.melo@example.com         | (61) 4446-4868 |    51 |
|  1 | Consulino    | Gonçalves   | male     | consulino.goncalves@example.com | (48) 5295-3687 |    33 |
|  2 | Geneci       | da Mata     | female   | geneci.damata@example.com       | (17) 2119-0986 |    36 |
|  3 | Indiara      | Vieira      | female   | indiara.vieira@example.com      | (58) 8267-5360 |    63 |
|  4 | Lívian       | Costa       | female   | livian.costa@example.com        | (42) 5022-2854 |    46 |


In [35]:
"""
Tarefa 1.2

Crie uma coluna que indique o Estado do usuário, baseado em seu telefone celular
Se o telefone celular tiver um DDD inexistente remova o usuário da tabela
"""

def get_state_from_ddd(ddd):
    state = {
        "11": "São Paulo",
        "12": "São Paulo",
        "13": "São Paulo",
        "14": "São Paulo",
        "15": "São Paulo",
        "16": "São Paulo",
        "17": "São Paulo",
        "18": "São Paulo",
        "19": "São Paulo",
        "21": "Rio de Janeiro",
        "22": "Rio de Janeiro",
        "24": "Rio de Janeiro",
        "27": "Espírito Santo",
        "28": "Espírito Santo",
        "31": "Minas Gerais",
        "32": "Minas Gerais",
        "33": "Minas Gerais",
        "34": "Minas Gerais",
        "35": "Minas Gerais",
        "37": "Minas Gerais",
        "38": "Minas Gerais",
        "41": "Paraná",
        "42": "Paraná",
        "43": "Paraná",
        "44": "Paraná",
        "45": "Paraná",
        "46": "Paraná",
        "47": "Santa Catarina",
        "48": "Santa Catarina",
        "49": "Santa Catarina",
        "51": "Rio Grande do Sul",
        "53": "Rio Grande do Sul",
        "54": "Rio Grande do Sul",
        "55": "Rio Grande do Sul",
        "61": "Distrito Federal",
        "62": "Goiás",
        "63": "Tocantins",
        "64": "Goiás",
        "65": "Mato Grosso",
        "66": "Mato Grosso",
        "67": "Mato Grosso do Sul",
        "68": "Acre",
        "69": "Rondônia",
        "71": "Bahia",
        "73": "Bahia",
        "74": "Bahia",
        "75": "Bahia",
        "77": "Bahia",
        "79": "Sergipe",
        "81": "Pernambuco",
        "82": "Alagoas",
        "83": "Paraíba",
        "84": "Rio Grande do Norte",
        "85": "Ceará",
        "86": "Piauí",
        "87": "Pernambuco",
        "88": "Ceará",
        "89": "Piauí",
        "91": "Pará",
        "92": "Amazonas",
        "93": "Pará",
        "94": "Pará",
        "95": "Roraima",
        "96": "Amapá",
        "97": "Amazonas",
        "98": "Maranhão",
        "99": "Maranhão"
    }
    return state.get(ddd, None)

for index, user_row in users_df.iterrows():
    ddd = user_row.cell[1:3]
    state = get_state_from_ddd(ddd)

    if state:
        users_df.loc[index, "state"] = state
    else:
        users_df.drop(index, inplace=True)

users_df.reset_index(drop=True, inplace=True)

print(users_df.head().to_markdown())


|    | first_name   | last_name   | gender   | email                           | cell           |   age | state            |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|:-----------------|
|  0 | Jovina       | Melo        | female   | jovina.melo@example.com         | (61) 4446-4868 |    51 | Distrito Federal |
|  1 | Consulino    | Gonçalves   | male     | consulino.goncalves@example.com | (48) 5295-3687 |    33 | Santa Catarina   |
|  2 | Geneci       | da Mata     | female   | geneci.damata@example.com       | (17) 2119-0986 |    36 | São Paulo        |
|  3 | Lívian       | Costa       | female   | livian.costa@example.com        | (42) 5022-2854 |    46 | Paraná           |
|  4 | Renata       | Cardoso     | female   | renata.cardoso@example.com      | (98) 9721-5516 |    57 | Maranhão         |


In [36]:
"""
Tarefa 2

Baixe os Pokémons iniciais de todas as gerações por meio da PokeAPI
"""

initial_pokemons = [
    "bulbasaur",
    "charmander",
    "squirtle",
    "pikachu",
    "chikorita",
    "cyndaquil",
    "totodile",
    "treecko",
    "torchic",
    "mudkip",
    "turtwig",
    "chimchar",
    "piplup",
    "snivy",
    "tepig",
    "oshawott",
    "chespin",
    "fennekin",
    "froakie",
    "rowlet",
    "litten",
    "popplio",
    "grookey",
    "scorbunny",
    "sobble",
]

url = "https://pokeapi.co/api/v2/pokemon/"

pokemons = []
for pokemon_name in initial_pokemons:
    url_pokemon = url + f"{pokemon_name}"
    response = requests.get(url_pokemon)
    if response.status_code == 200:
        pokemons.append(response.json())
        print(f"{pokemon_name} - {response}")
    else:
        print(f"{pokemon_name} - {response}")


bulbasaur - <Response [200]>
charmander - <Response [200]>
squirtle - <Response [200]>
pikachu - <Response [200]>
chikorita - <Response [200]>
cyndaquil - <Response [200]>
totodile - <Response [200]>
treecko - <Response [200]>
torchic - <Response [200]>
mudkip - <Response [200]>
turtwig - <Response [200]>
chimchar - <Response [200]>
piplup - <Response [200]>
snivy - <Response [200]>
tepig - <Response [200]>
oshawott - <Response [200]>
chespin - <Response [200]>
fennekin - <Response [200]>
froakie - <Response [200]>
rowlet - <Response [200]>
litten - <Response [200]>
popplio - <Response [200]>
grookey - <Response [200]>
scorbunny - <Response [200]>
sobble - <Response [200]>


In [37]:
"""
Tarefa 2.1

Atribua um Pokémon inicial aleatório para cada um dos usuários
"""

for user_index, user_row in users_df.iterrows():
    users_df.loc[user_index, "pokemon"] = random.choice(initial_pokemons)

print(users_df.head().to_markdown())


|    | first_name   | last_name   | gender   | email                           | cell           |   age | state            | pokemon   |
|---:|:-------------|:------------|:---------|:--------------------------------|:---------------|------:|:-----------------|:----------|
|  0 | Jovina       | Melo        | female   | jovina.melo@example.com         | (61) 4446-4868 |    51 | Distrito Federal | snivy     |
|  1 | Consulino    | Gonçalves   | male     | consulino.goncalves@example.com | (48) 5295-3687 |    33 | Santa Catarina   | cyndaquil |
|  2 | Geneci       | da Mata     | female   | geneci.damata@example.com       | (17) 2119-0986 |    36 | São Paulo        | rowlet    |
|  3 | Lívian       | Costa       | female   | livian.costa@example.com        | (42) 5022-2854 |    46 | Paraná           | oshawott  |
|  4 | Renata       | Cardoso     | female   | renata.cardoso@example.com      | (98) 9721-5516 |    57 | Maranhão         | piplup    |


In [38]:
"""
Tarefa 3.a

Conecte com o Google Drive
Disponível em: https://drive.google.com/drive/folders/1GvSdaQwIXeNE1KBhb-3Z-r-Sp_fCT2-E?usp=sharing
"""

SCOPES = ["https://www.googleapis.com/auth/drive"]
credentials_json = Path("..") / "credentials.json"
flow = InstalledAppFlow.from_client_secrets_file(credentials_json, SCOPES)
with io.capture_output() as captured:
    credentials = flow.run_local_server(port=0)
service = build("drive", "v3", credentials=credentials)


In [76]:
"""
Tarefa 3.b

Crie uma pasta para cada elemento dos Pokémons
"""

pokemon_types = []
for pokemon in pokemons:
    for type_item in pokemon["types"]:
        pokemon_types.append(type_item["type"]["name"])
pokemon_types = list(set(pokemon_types))


env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

greenpeace_folder_id = os.environ.get("GOOGLE_DRIVE_GREENPEACE_FOLDER_ID")
try:
    files = service.files().list(q=f"'{greenpeace_folder_id}' in parents").execute()
except HttpError as err:
    print(err)

file_names_list = []
for file in files["files"]:
        file_names_list.append(file["name"])

google_drive_folder_id_by_type = {}
for pokemon_type in pokemon_types:
    if pokemon_type not in file_names_list:
        try:
            file_metadata = {
                "name": pokemon_type,
                "mimeType": "application/vnd.google-apps.folder",
                "parents": [greenpeace_folder_id],
            }
            file = service.files().create(body=file_metadata).execute()
            google_drive_folder_id_by_type[pokemon_type] = file.get('id')
            print(f"Folder for type *{pokemon_type}* created successfully with ID: {file.get('id')}")

        except HttpError as err:
            print(f"{err} for type *{pokemon_type}*")
    else:
        print(f"Folder for type *{pokemon_type}* already exists")


Folder for type *grass* already exists
Folder for type *electric* already exists
Folder for type *flying* already exists
Folder for type *fire* already exists
Folder for type *water* already exists
Folder for type *poison* already exists


In [90]:
"""
Tarefa 3.1

Adicione em cada uma das pastas dois arquivos em CSV
Cada pasta deve ter dois arquivos um com todos os homens com Pokémons
daquele tipo, e outro com todas as mulheres com Pokémons daquele tipo
"""

pokemon_by_type = {
    pokemon_type: [] for pokemon_type in pokemon_types
}

for pokemon in pokemons:
    for type_item in pokemon["types"]:
        for pokemon_type in pokemon_types:
            if type_item["type"]["name"] == pokemon_type:
                pokemon_by_type[pokemon_type].append(pokemon["name"])

for pokemon_type in pokemon_types:
    filtered_users_df = users_df[users_df["pokemon"].isin(pokemon_by_type[pokemon_type])]

    male_filtered_users_df = filtered_users_df[filtered_users_df["gender"] == "male"]
    male_filtered_users_df.to_csv(f"csvs/{pokemon_type}_male_users.csv", index=False)

    female_filtered_users_df = filtered_users_df[filtered_users_df["gender"] == "female"]
    female_filtered_users_df.to_csv(f"csvs/{pokemon_type}_female_users.csv", index=False)

